<a href="https://colab.research.google.com/github/27abernal/Adv_AI/blob/main/Data_Preprocessing_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================
# Clean → Encode → Classify (KNN) - Simplified
# ===============================================
# Dataset: Kaggle "Mushroom Classification"
# Expected file: ./mushrooms.csv
# Team Members: Alicia & Anya

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.impute import SimpleImputer  # used ONCE as an example
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, auc
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# -----------------------------
# 0) Load data
# -----------------------------
csv_path = "/content/mushrooms.csv"   # TODO: place the file or change path
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"Could not find {csv_path} — download from Kaggle and try again.")

df = pd.read_csv(csv_path)
print("Initial shape:", df.shape)
# TODO: Get the top 5 rows (1 line of code)
print(df.head())  # Gets the top 5 rows of the dataframe

Initial shape: (8124, 23)
  class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w       

In [ ]:
# -----------------------------
# 1) Basic cleaning (duplicates, empty rows)
# -----------------------------
before = df.shape[0]

# TODO: Remove duplicates (1 line of code)
df.drop_duplicates(inplace=True)  # Remove duplicate rows from the dataset

print("Removed duplicates:", before - df.shape[0])

before = df.shape[0]
# TODO: Drop fully empty rows (1 line of code)
df.dropna(how='all', inplace=True)  # Remove rows where ALL values are missing

print("Dropped fully empty rows:", before - df.shape[0])

print("\nMissing values per column (top 10):")
# TODO: Display the missing values per column (1 line of code)
print(df.isna().sum().head(10))  # Display top 10 columns with missing values

Removed duplicates: 0
Dropped fully empty rows: 0

Missing values per column (top 10):
class              0
cap-shape          0
cap-surface        0
cap-color          0
bruises            0
odor               0
gill-attachment    0
gill-spacing       0
gill-size          0
gill-color         0
dtype: int64


In [ ]:
# -----------------------------
# 2) Target & split
# -----------------------------
TARGET_COL = "class"  # TODO: change if your label column differs
if TARGET_COL not in df.columns:
    raise ValueError("Update TARGET_COL to match your dataset's label column.")

# TODO: Drop the target column from the "X" (1 line of code)
X = df.drop(columns=[TARGET_COL])  # Create feature matrix by removing target column

# TODO: Set up the target column (1 line of code)
y = df[TARGET_COL]  # Create target vector containing only the class labels

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y
)
print("\nTrain/Test shapes:", X_train.shape, X_test.shape)

# Detect types BEFORE imputation/encoding
# TODO: Extract the numeric columns from the training data set and store it as a list
orig_numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
# TODO: Extract the categorical columns from the training data set and store it as a list
orig_categorical_cols = X.select_dtypes(exclude=['number']).columns.tolist()
print("Original numeric cols:", orig_numeric_cols if orig_numeric_cols else "None")
print("Original categorical cols (sample):", orig_categorical_cols[:10])


Train/Test shapes: (6499, 22) (1625, 22)
Original numeric cols: None
Original categorical cols (sample): ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape']


In [ ]:
# --------------------------------------------
# 3) IMPUTATION (MANUAL) + one sklearn example
# --------------------------------------------
# Manual imputation rules:
#  - Numeric: fill with TRAIN median
#  - Categorical: fill with TRAIN mode
# NOTE: We compute imputation values on TRAIN only, then apply to both TRAIN & TEST.

X_train_imp = X_train.copy()
X_test_imp  = X_test.copy()

# a) Numeric → manual median
# TODO: For each of the columns, fill the empty values with the median of the column (4 lines of code)
# TODO: Do this for the training and test data separately.
for col in X_train_imp.select_dtypes(include=['number']).columns:  # Loop through numeric columns
    X_train_imp[col] = X_train_imp[col].fillna(X_train_imp[col].median())  # Fill missing values with column median in training data
    X_test_imp[col] = X_test_imp[col].fillna(X_train_imp[col].median())  # Fill missing values with TRAIN median in test data

# b) Categorical → manual mode
# TODO: For each of the columns, fill the empty values with the mode of the column (4-5 lines of code)
# TODO: Do this for the training and test data separately.
for col in X_train_imp.select_dtypes(exclude=['number']).columns:  # Loop through categorical columns
    X_train_imp[col] = X_train_imp[col].fillna(X_train_imp[col].mode()[0])  # Fill missing values with most frequent value in training data
    X_test_imp[col] = X_test_imp[col].fillna(X_train_imp[col].mode()[0])  # Fill missing values with TRAIN mode in test data

# c) One EXAMPLE using sklearn SimpleImputer on a SINGLE column (categorical)
# Choose a categorical column present in your dataset:
IMPUTE_EXAMPLE_COL = "odor"  # TODO: change if needed
if IMPUTE_EXAMPLE_COL in X_train_imp.columns:
    cat_imp = SimpleImputer(strategy="most_frequent")
    # Ensure the input is 2D by selecting the column as a DataFrame (not a Series)
    X_train_imp[IMPUTE_EXAMPLE_COL] = cat_imp.fit_transform(X_train_imp[[IMPUTE_EXAMPLE_COL]]).ravel()
    X_test_imp[IMPUTE_EXAMPLE_COL] = cat_imp.transform(X_test_imp[[IMPUTE_EXAMPLE_COL]]).ravel()
    print(f"\nUsed sklearn SimpleImputer on column: {IMPUTE_EXAMPLE_COL}")
else:
    print(f"\n[Note] Example imputer column '{IMPUTE_EXAMPLE_COL}' not found. Skipping sklearn example.")


Used sklearn SimpleImputer on column: odor


In [ ]:
# --------------------------------------------
# 4) ENCODING
#    - EXPLICIT One-Hot Encoding on ONE chosen column
#    - Then get_dummies for the remaining categoricals
# --------------------------------------------
# Choose the column for explicit OHE:
EXPLICIT_OHE_COL = "odor"  # TODO: change if needed

X_train_enc = X_train_imp.copy()
X_test_enc  = X_test_imp.copy()

if EXPLICIT_OHE_COL in X_train_enc.columns:
    # EXPLICIT OHE on this ONE column — fit on TRAIN only and TEST only separately (2 lines of code)
    # Align test columns to train (avoid unseen-category issues)
    train_ohe = pd.get_dummies(X_train_enc[EXPLICIT_OHE_COL], prefix=EXPLICIT_OHE_COL)  # Convert categorical column to dummy variables for training data
    test_ohe = pd.get_dummies(X_test_enc[EXPLICIT_OHE_COL], prefix=EXPLICIT_OHE_COL)  # Convert categorical column to dummy variables for test data
    test_ohe = test_ohe.reindex(columns=train_ohe.columns, fill_value=0)  # Ensure test has same columns as train, filling missing with 0

    # Drop original column and concat OHE columns
    X_train_enc = pd.concat([X_train_enc.drop(columns=[EXPLICIT_OHE_COL]), train_ohe], axis=1)  # Remove original column and add dummy columns for training
    X_test_enc  = pd.concat([X_test_enc.drop(columns=[EXPLICIT_OHE_COL]),  test_ohe], axis=1)  # Remove original column and add dummy columns for test

    print(f"\nExplicit OHE applied to column: {EXPLICIT_OHE_COL}")
else:
    print(f"\n[Note] Explicit OHE column '{EXPLICIT_OHE_COL}' not found. Skipping explicit OHE step.")

# TODO: Identify remaining categorical columns (post explicit OHE drop) in the TRAINING data set only (1 line of code)
remaining_categorical_cols = X_train_enc.select_dtypes(include=['object']).columns.tolist()  # Find all remaining categorical columns after explicit OHE

# TODO: Apply get_dummies to the remaining categoricals (fit on TRAIN, align TEST) (1 lines of code)
X_train_enc = pd.get_dummies(X_train_enc, columns=remaining_categorical_cols)  # Automatically convert all remaining categorical columns to dummy variables

# Align test columns to train columns (VERY IMPORTANT)
X_test_enc = X_test_enc.reindex(columns=X_train_enc.columns, fill_value=0)

print("Encoded train shape:", X_train_enc.shape)
print("Encoded test shape:", X_test_enc.shape)


Explicit OHE applied to column: odor
Encoded train shape: (6499, 117)
Encoded test shape: (1625, 117)


In [ ]:
# --------------------------------------------
# 5) NORMALIZE numeric columns (if any)
#    (Scale ONLY the original numeric columns; don't scale one-hot columns)
# --------------------------------------------
# TODO: Use the Standard Scaler to scale the numeric values (1 line of code)
scaler = StandardScaler()  # Create StandardScaler instance to standardize features (mean=0, std=1)

numeric_cols_to_scale = [c for c in orig_numeric_cols if c in X_train_enc.columns]

# TODO: Fit the scaler and scale the train and test data separately. (3 lines of code)
# TODO: Print the names of the columns scaled if any (4 lines of code at max)
if numeric_cols_to_scale:
    X_train_enc[numeric_cols_to_scale] = scaler.fit_transform(X_train_enc[numeric_cols_to_scale])  # Fit scaler on training data and transform training numeric columns
    X_test_enc[numeric_cols_to_scale] = scaler.transform(X_test_enc[numeric_cols_to_scale])  # Transform test numeric columns using scaler fitted on training data
    print(f"\nScaled numeric columns: {numeric_cols_to_scale}")  # Print which columns were scaled
else:
    print("No original numeric columns to scale.")  # Inform user if no numeric columns were found to scale

No original numeric columns to scale.


In [ ]:
# --------------------------------------------
# 6) KNN + Grid Search
# --------------------------------------------
# TODO: Use the KNN Classifier to classify the data
knn = KNeighborsClassifier()  # Create KNN classifier instance for grid search
# TODO: Test for number of neighbors in the set [3, 5, 7, 9, 11, 15, 21]
# TODO: Try both the types of weights
# TODO: Also try Manhattan and Euclidean distances.
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 21],  # Test different k values (number of neighbors)
    'weights': ['uniform', 'distance'],  # Test both uniform and distance-based weighting
    'p': [1, 2]  # 1: Manhattan, 2: Euclidean
}
# Find and print the best_estimator_ and print the best parameters
# Roughly 7 - 9 lines of code including printing the estimator.
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)  # Create grid search with 5-fold cross-validation
grid_search.fit(X_train_enc, y_train)  # Perform grid search on training data
print(f"\nBest CV accuracy: {grid_search.best_score_:.4f}")  # Print best cross-validation accuracy
print(f"Best params: {grid_search.best_params_}")  # Print best hyperparameters found
print(f"Best estimator: {grid_search.best_estimator_}")  # Print the best configured KNN model
best_knn = grid_search.best_estimator_  # Store the best model for later use

In [ ]:
# --------------------------------------------
# 7) Evaluate on held-out TEST
# --------------------------------------------
# TODO: Use KNN to make predictions on the test dataset (1 line of code)
y_pred = best_knn.predict(X_test_enc)  # Generate predictions on test set using best KNN model

test_acc = accuracy_score(y_test, y_pred)
print("\nTest accuracy: {:.4f}".format(test_acc))
print("\nClassification report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred, labels=np.unique(y))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y))
plt.figure()
disp.plot(values_format="d")
plt.title("Confusion Matrix (Test)")
plt.tight_layout()
plt.show()